<a href="https://colab.research.google.com/github/AP2324/trabajo-equipo-9/blob/main/03_data_scientist_download_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import syft as sy
import jax
from jax.example_libraries import stax
from jax.example_libraries.stax import Dense, Relu, LogSoftmax
from jax import random
import jax.numpy as jnp
from mnist_dataset import mnist

In [ ]:
# Lanza un nodo de PySyft llamado 'mnist-domain' en modo de desarrollo.
node = sy.orchestra.launch(name="mnist-domain", dev_mode=True)
# El DS accede a los conjuntos de datos disponibles en el dominio.
ds_client = node.login(email="sheldon17@caltech.edu", password="changethis")

Staging Protocol Changes...
Data Migrated to latest version !!!
Logged into <mnist-domain: High side Domain> as <sheldon17@caltech.edu>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

## Después de que el DO haya ejecutado el código y depositado los resultados, el DS los descarga

In [ ]:
# Accede a los activos dentro del conjunto de datos; en este caso, imágenes y etiquetas de entrenamiento.
datasets = ds_client.datasets.get_all()
assets = datasets[0].assets
assert len(assets) == 2 # Asegura que hay dos activos: imágenes y etiquetas.

In [ ]:
# Extrae los punteros a las imágenes y etiquetas de entrenamiento
training_images = assets[0]
training_labels = assets[1]

In [ ]:
# Accede al espacio de código del Data Scientist dentro del dominio PySyft.
# Esto permite al DS ver y gestionar el código que ha enviado para ejecución,
# incluyendo solicitudes de código, funciones definidas y cualquier resultado de código ejecutado por el DO.
ds_client.code

In [ ]:
# El DS recupera los resultados del entrenamiento ejecutado por el DO.
result = ds_client.code.mnist_3_linear_layers(mnist_images=training_images, mnist_labels=training_labels)

In [ ]:
# Recupera los resultados del entrenamiento ejecutado por el DO desde el dominio PySyft.
# 'train_accs' contiene las precisiones de entrenamiento para cada época, mostrando cómo mejoró el modelo a lo largo del tiempo.
# 'params' contiene los parámetros finales del modelo (pesos y sesgos) después de completar el entrenamiento.
train_accs, params = result.get_from(ds_client)


In [ ]:
assert isinstance(train_accs, list)
train_accs

[Array(0.99700004, dtype=float32)]

In [ ]:
# Verifica que los parámetros del modelo ('params') recuperados sean una lista, asegurando que la estructura de datos es la esperada.
assert isinstance(params, list)

# Utiliza 'jax.tree_map' para aplicar una función lambda a cada elemento en la estructura de 'params'.
# La función lambda extrae la forma ('shape') de cada parámetro del modelo, proporcionando una visión clara de las dimensiones de los parámetros.
jax.tree_map(lambda x: x.shape, params)


[((784, 1024), (1024,)),
 (),
 ((1024, 1024), (1024,)),
 (),
 ((1024, 10), (10,)),
 ()]

## Teniendo los pesos entrenados, el DS puede realizar inferencias en su conjunto de datos de prueba MNIST

In [ ]:
# Carga el conjunto de datos de prueba MNIST.
# Esta operación extrae únicamente las imágenes y etiquetas de prueba,
# omitiendo los conjuntos de datos de entrenamiento mediante el uso de '_'.
# Se asume que este conjunto de datos de prueba es accesible y puede ser utilizado por el DS
# para evaluar el rendimiento del modelo en datos no vistos.
_, _, test_images, test_labels = mnist()

_, _, test_images, test_labels = mnist()

In [ ]:
assert test_images.shape == (10000, 784)  # Asegura que las dimensiones de las imágenes de prueba son correctas.
assert test_labels.shape == (10000, 10)  # Asegura que las dimensiones de las etiquetas de prueba son correctas.


#### Define the neural network and the accuracy function

In [ ]:
# Inicializa la arquitectura de la red neuronal utilizando la API stax de JAX.
# La red consta de una secuencia de capas definida por 'stax.serial':
# 1. Una capa densa con 1024 unidades seguida de una función de activación ReLU.
# 2. Otra capa densa con 1024 unidades seguida de una función de activación ReLU.
# 3. Una capa de salida densa con 10 unidades (una por cada clase de dígito MNIST)
#    que utiliza LogSoftmax para obtener las probabilidades logarítmicas.
# 'init_random_params' se utilizará para inicializar los parámetros de la red de forma aleatoria,
# y 'predict' será la función para realizar predicciones con la red.
init_random_params, predict = stax.serial(
    Dense(1024), Relu,
    Dense(1024), Relu,
    Dense(10), LogSoftmax)



In [ ]:
# Define una función para calcular la precisión del modelo.
# La precisión se calcula como el promedio de predicciones correctas.
# Parámetros:
# - 'params': los parámetros actuales de la red neuronal (pesos y sesgos).
# - 'batch': un lote de datos que contiene 'inputs' (imágenes) y 'targets' (etiquetas verdaderas).
# Proceso:
# 1. 'inputs' y 'targets' se extraen del lote.
# 2. Se calcula la clase predicha por el modelo para cada entrada utilizando 'predict'.
# 3. Se identifica la clase real de cada etiqueta mediante 'argmax', que devuelve el índice de la máxima probabilidad.
# 4. Se calcula la precisión como el promedio de coincidencias entre clases predichas y reales.
def accuracy(params, batch):
    inputs, targets = batch
    target_class = jnp.argmax(targets, axis=1)  # Clase real
    predicted_class = jnp.argmax(predict(params, inputs), axis=1)  # Clase predicha
    return jnp.mean(predicted_class == target_class)  # Promedio de aciertos


#### Test de inferencia usando pesos aleatorios.

In [ ]:
# Genera una clave de números pseudoaleatorios para inicializar los parámetros de la red.
rng = random.PRNGKey(0)

# Inicializa los parámetros de la red de forma aleatoria para evaluar el comportamiento base del modelo.
# La forma de entrada (-1, 28 * 28) indica que la red espera imágenes de 28x28 píxeles aplanadas.
_, random_params = init_random_params(rng, (-1, 28 * 28))

# Evalúa la precisión del modelo con pesos aleatorios en el conjunto de datos de prueba MNIST.
# Esto proporciona una línea base para entender el rendimiento del modelo sin entrenamiento.
test_acc = accuracy(random_params, (test_images, test_labels))

# Imprime la precisión del conjunto de prueba utilizando los pesos aleatorios,
# convirtiéndola en un porcentaje para una interpretación más clara.
print(f"Precisión del conjunto de prueba con pesos aleatorios = {test_acc * 100 : .2f}%")


Precisión del conjunto de prueba con pesos aleatorios =  8.68%


#### Prueba de inferencia utilizando los pesos entrenados recibidos del DO.

In [ ]:
# Evalúa la precisión del modelo en el conjunto de datos de prueba MNIST utilizando los pesos entrenados recibidos del DO.
# 'params' contiene los pesos y sesgos optimizados del modelo después del entrenamiento.
# 'test_images' y 'test_labels' son las imágenes y etiquetas del conjunto de prueba, respectivamente.
test_acc = accuracy(params, (test_images, test_labels))

# Imprime la precisión del conjunto de prueba, convirtiendo el valor a porcentaje para facilitar su interpretación.
# Esta precisión refleja el rendimiento del modelo en datos no vistos después de haber sido entrenado,
# proporcionando una medida de la capacidad del modelo para generalizar.
print(f"Precisión del conjunto de prueba con pesos entrenados = {test_acc * 100 : .2f}%")


Precisión del conjunto de prueba con pesos entrenados =  88.08%


In [ ]:
# Evalúa la precisión del modelo en el conjunto de datos de prueba MNIST utilizando los pesos entrenados recibidos del DO.
# 'params' contiene los pesos y sesgos optimizados del modelo después del entrenamiento.
# 'test_images' y 'test_labels' son las imágenes y etiquetas del conjunto de prueba, respectivamente.
test_acc = accuracy(params, (test_images, test_labels))

# Imprime la precisión del conjunto de prueba, convirtiendo el valor a porcentaje para facilitar su interpretación.
# Esta precisión refleja el rendimiento del modelo en datos no vistos después de haber sido entrenado,
# proporcionando una medida de la capacidad del modelo para generalizar.
print(f"Precisión del conjunto de prueba con pesos entrenados = {test_acc * 100 : .2f}%")


Precisión del conjunto de prueba con pesos entrenados =  88.08%


In [ ]:
test_acc = accuracy(params, (test_images, test_labels))
print(f"Test set accuracy with trained weights = {test_acc * 100 : .2f}%")

Test set accuracy with trained weights =  88.08%


La precisión del 88.08% alcanzada por el modelo en el conjunto de datos de prueba MNIST, tras haber sido entrenado con los pesos proporcionados por el Data Owner (DO), refleja un nivel de rendimiento notablemente bueno, indicativo de que el modelo ha logrado captar y aprender eficazmente los patrones subyacentes en los datos de entrenamiento. Este nivel de precisión demuestra que el modelo no solo ha asimilado las características específicas de los datos de entrenamiento, sino que también ha desarrollado la capacidad de generalizar correctamente a nuevos datos, un aspecto fundamental para la aplicación práctica de modelos de aprendizaje automático. El hecho de que el modelo logre una precisión significativamente superior a lo que se esperaría de un modelo con pesos inicializados de manera aleatoria subraya la eficacia del entrenamiento y el aprendizaje que ha tenido lugar.

Sin embargo, es importante reconocer que, aunque el resultado es prometedor, aún existe margen para la mejora y optimización del modelo. Estrategias adicionales como el ajuste meticuloso de hiperparámetros, la implementación de técnicas avanzadas de regularización como la normalización por lotes, y el aprovechamiento de métodos de aumento de datos podrían contribuir a elevar aún más la precisión del modelo. Estas técnicas pueden ayudar a mitigar el sobreajuste, mejorar la robustez del modelo y aumentar su capacidad para captar la variabilidad inherente a los datos de entrada.
